# TetBot AI:
## Genetic Algorithm solution for Tetris
#### by Sofie Tien, Stephan Bork, Constantin Phillipi and Robert Richartz

In [ ]:
# Zu allem notizen machen....
# Den ganzen Code Erklären....
# Add Pictures

Sources:

### Was ist Tetris?
points, how the game works, invention etc...
Warum haben wir uns für Tetris entschieden?

### Was ist ein Genetic Algorithm?

### Wie kann man Tetris Optimieren?

---> Hier das Ganze Konzept erklären
---> How to Optimize
---> How does the Algorithm Work.....
---> Welche kennzahlen benutzen und warum (Kennzahlen Erläutern)

---> Problem in creation...

---> Fittness Function erklären (wieso diese Funktioniert)
---> Breeding/Tournament Selection

---> Data Obtained

---> Next possible steps/Optimizations

#### Import Dependancies
Explanation...

In [1]:
# GIT HELP
# https://blog.reviewnb.com/github-jupyter-notebook/
# https://docs.github.com/en/get-started/getting-started-with-git/caching-your-github-credentials-in-git
# https://github.com/cli/cli#installation

# Create repo online in git hup and get the url
!git help

usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           <command> [<args>]

These are common Git commands used in various situations:

start a working area (see also: git help tutorial)
   clone     Clone a repository into a new directory
   init      Create an empty Git repository or reinitialize an existing one

work on the current change (see also: git help everyday)
   add       Add file contents to the index
   mv        Move or rename a file, a directory, or a symlink
   restore   Restore working tree files
   rm        Remove files from the working tree and from the index

examine the history and state (see also: git help revisions)
   bisect    Use binary search to find the commit that introduced a bug
   diff      Show changes between c

In [ ]:
import pygame  # version 1.9.3
import random
import math
import sys
import numpy as np
import csv

Base Game setup

Set Up Environment and add methods/functions to allows the Agent to act within the environment (explain these)

explanation...

In [ ]:
pygame.init()
pygame.font.init()

DISPLAY_WIDTH = 800
DISPLAY_HEIGHT = 600

gameDisplay = pygame.display.set_mode((DISPLAY_WIDTH, DISPLAY_HEIGHT))
pygame.display.set_caption('Tetris')
clock = pygame.time.Clock()

pieceNames = ('I', 'O', 'T', 'S', 'Z', 'J', 'L')

# Dictionary which assigns numerical values to the piece types for easier interpretation in the matrix
pieceValue = {'I': 1, 'O': 2, 'T': 3, 'S': 4, 'Z': 5, 'J': 6, 'L': 7}

# Inverse of the pieceValue dictionary
inv_pieceValue = {v: k for k, v in pieceValue.items()}

STARTING_LEVEL = 0  # Change this to start a new game at a higher level

MOVE_PERIOD_INIT = 4  # Piece movement speed when up/right/left arrow keys are pressed (Speed is defined as frame count. Game is 60 fps)

CLEAR_ANI_PERIOD = 4  # Line clear animation speed
SINE_ANI_PERIOD = 120  # Sine blinking effect speed

# Font sizes
SB_FONT_SIZE = 29
FONT_SIZE_SMALL = 17
PAUSE_FONT_SIZE = 66
GAMEOVER_FONT_SIZE = 66
TITLE_FONT_SIZE = 70
VERSION_FONT_SIZE = 20

fontSB = pygame.font.SysFont('agencyfb', SB_FONT_SIZE)
fontSmall = pygame.font.SysFont('agencyfb', FONT_SIZE_SMALL)
fontPAUSE = pygame.font.SysFont('agencyfb', PAUSE_FONT_SIZE)
fontGAMEOVER = pygame.font.SysFont('agencyfb', GAMEOVER_FONT_SIZE)
fontTitle = pygame.font.SysFont('agencyfb', TITLE_FONT_SIZE)
fontVersion = pygame.font.SysFont('agencyfb', VERSION_FONT_SIZE)

ROW = (0)
COL = (1)

# Some color definitions
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
DARK_GRAY = (80, 80, 80)
GRAY = (110, 110, 110)
LIGHT_GRAY = (150, 150, 150)
BORDER_COLOR = GRAY
NUM_COLOR = WHITE
TEXT_COLOR = GRAY

blockColors = {
    'I': (19, 232, 232),  # CYAN
    'O': (236, 236, 14),  # YELLOW
    'T': (126, 5, 126),  # PURPLE
    'S': (0, 128, 0),  # GREEN
    'Z': (236, 14, 14),  # RED
    'J': (30, 30, 201),  # BLUE
    'L': (240, 110, 2)}  # ORANGE

# Initial(spawn) block definitons of each piece
pieceDefs = {
    'I': ((1, 0), (1, 1), (1, 2), (1, 3)),
    'O': ((0, 1), (0, 2), (1, 1), (1, 2)),
    'T': ((0, 1), (1, 0), (1, 1), (1, 2)),
    'S': ((0, 1), (0, 2), (1, 0), (1, 1)),
    'Z': ((0, 0), (0, 1), (1, 1), (1, 2)),
    'J': ((0, 0), (1, 0), (1, 1), (1, 2)),
    'L': ((0, 2), (1, 0), (1, 1), (1, 2))}

directions = {
    'down': (1, 0),
    'right': (0, 1),
    'left': (0, -1),
    'downRight': (1, 1),
    'downLeft': (1, -1),
    'noMove': (0, 0)}

# TODO - Make it so that blocks can only be soft dropped (all speed = 999)
levelSpeeds = (9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999)

# The speed of the moving piece at each level. Level speeds are defined as levelSpeeds[level]
# Each 10 cleared lines means a level up.
# After level 29, speed is always 1. Max level is 99

baseLinePoints = (0, 40, 100, 300, 1200)


# Total score is calculated as: Score = level*baseLinePoints[clearedLineNumberAtATime] + totalDropCount
# Drop means the action the player forces the piece down instead of free fall(By key combinations: down, down-left, down-rigth arrows)


Need GameKeyInput to make the Pygame Module to work

In [ ]:
# Class for the game input keys and their status
class GameKeyInput:

    def __init__(self):
        self.xNav = self.KeyName('idle', False)  # 'left' 'right'
        self.down = self.KeyName('idle', False)  # 'pressed' 'released'
        self.rotate = self.KeyName('idle', False)  # 'pressed' //KEY UP
        self.cRotate = self.KeyName('idle', False)  # 'pressed' //KEY Z
        self.enter = self.KeyName('idle', False)  # 'pressed' //KEY Enter
        self.pause = self.KeyName('idle', False)  # 'pressed' //KEY P
        self.restart = self.KeyName('idle', False)  # 'pressed' //KEY R

        # Code a hard drop
        self.hardDrop = self.KeyName('idle', False)  # 'pressed' //KEY SPACE

    class KeyName:

        def __init__(self, initStatus, initTrig):
            self.status = initStatus
            self.trig = initTrig

GameClock Class allows us to make use of Timing within the

In [ ]:
# Class for the game's timing events
class GameClock:

    def __init__(self):
        self.frameTick = 0  # The main clock tick of the game, increments at each frame (1/60 secs, 60 fps) 
        
        # For AI Timing increased to 200 Fps (For faster computation...)
        
        self.pausedMoment = 0
        self.move = self.TimingType(MOVE_PERIOD_INIT)  # Drop and move(right and left) timing object
        self.fall = self.TimingType(levelSpeeds[STARTING_LEVEL])  # Free fall timing object
        self.clearAniStart = 0

    class TimingType:

        def __init__(self, framePeriod):
            self.preFrame = 0
            self.framePeriod = framePeriod

        def check(self, frameTick):
            if frameTick - self.preFrame > self.framePeriod - 1:
                self.preFrame = frameTick
                return True
            return False

    def pause(self):
        self.pausedMoment = self.frameTick

    def unpause(self):
        self.frameTick = self.pausedMoment

    def restart(self):
        self.frameTick = 0
        self.pausedMoment = 0
        self.move = self.TimingType(MOVE_PERIOD_INIT)
        self.fall = self.TimingType(levelSpeeds[STARTING_LEVEL])
        self.clearAniStart = 0

    def update(self):
        self.frameTick = self.frameTick + 1

Mainboard Class
Holds all information of the Tetris field

2 Very important Attributes within this class

* Blockmat --> Matrix representation of the Playing field. The Blockmat from the base source code consists of strings. We will later convert this into a numpy array for faster computation. This is used to allow the agent to interpret the environment

* Piece with Subclass blocks. This class gives us the status (moving, collided, uncreated) of the Tetris piece which is currently in play. By having access to this object we are able to move the piece within the limitations of the game (without worrying about array out of bounds or overlapping pieces) and also access the position of the piece by looking at the individual positions of the blocks

In [ ]:
# Class for all the game mechanics, visuals and events
class MainBoard:

    def __init__(self, blockSize, xPos, yPos, colNum, rowNum, boardLineWidth, blockLineWidth, scoreBoardWidth):

        # Size and position initiations
        self.blockSize = blockSize
        self.xPos = xPos
        self.yPos = yPos
        self.colNum = colNum
        self.rowNum = rowNum
        self.boardLineWidth = boardLineWidth
        self.blockLineWidth = blockLineWidth
        self.scoreBoardWidth = scoreBoardWidth

        self.position_values = np.full((4, 10), np.NINF)

        # Matrix that contains all the existing blocks in the game board, except the moving piece
        self.blockMat = [['empty'] * colNum for i in range(rowNum)]

        self.piece = MovingPiece(colNum, rowNum, 'uncreated')

        self.lineClearStatus = 'idle'  # 'clearRunning' 'clearFin'
        self.clearedLines = [-1, -1, -1, -1]

        self.gameStatus = 'firstStart'  # 'running' 'gameOver'
        self.gamePause = False
        self.nextPieces = ['I', 'I']

        self.score = 0
        self.level = STARTING_LEVEL
        self.lines = 0

    def get_level(self):
        return self.level

    def get_height(self):
        return self.lines

    def get_mainBoard_pos(self, row, col):
        return self.blockMat[row][col]

    # Assisstance Functions for the AI
    
    #  Help to create the blockMat for the next piece (look ahead piece)
    def set_blockmat(self, new_blockmat):
        self.blockMat = new_blockmat
        
    # Help to set the piece of the next blockMat
    def set_piece(self, new_piece):
        self.piece = new_piece

    def restart(self):
        self.blockMat = [['empty'] * self.colNum for i in range(self.rowNum)]

        self.piece = MovingPiece(self.colNum, self.rowNum, 'uncreated')

        self.lineClearStatus = 'idle'
        self.clearedLines = [-1, -1, -1, -1]
        gameClock.fall.preFrame = gameClock.frameTick
        self.generateNextTwoPieces()
        self.gameStatus = 'running'
        self.gamePause = False

        self.score = 0
        self.level = STARTING_LEVEL
        self.lines = 0

        gameClock.restart()

    def get_piece(self):
        return self.piece

    def erase_BLOCK(self, xRef, yRef, row, col):
        pygame.draw.rect(gameDisplay, BLACK,
                         [xRef + (col * self.blockSize), yRef + (row * self.blockSize), self.blockSize, self.blockSize],
                         0)

    def draw_BLOCK(self, xRef, yRef, row, col, color):
        pygame.draw.rect(gameDisplay, BLACK,
                         [xRef + (col * self.blockSize), yRef + (row * self.blockSize), self.blockSize,
                          self.blockLineWidth], 0)
        pygame.draw.rect(gameDisplay, BLACK, [xRef + (col * self.blockSize) + self.blockSize - self.blockLineWidth,
                                              yRef + (row * self.blockSize), self.blockLineWidth, self.blockSize], 0)
        pygame.draw.rect(gameDisplay, BLACK,
                         [xRef + (col * self.blockSize), yRef + (row * self.blockSize), self.blockLineWidth,
                          self.blockSize], 0)
        pygame.draw.rect(gameDisplay, BLACK, [xRef + (col * self.blockSize),
                                              yRef + (row * self.blockSize) + self.blockSize - self.blockLineWidth,
                                              self.blockSize, self.blockLineWidth], 0)

        pygame.draw.rect(gameDisplay, color, [xRef + (col * self.blockSize) + self.blockLineWidth,
                                              yRef + (row * self.blockSize) + self.blockLineWidth,
                                              self.blockSize - (2 * self.blockLineWidth),
                                              self.blockSize - (2 * self.blockLineWidth)], 0)

    def draw_GAMEBOARD_BORDER(self):
        pygame.draw.rect(gameDisplay, BORDER_COLOR, [self.xPos - self.boardLineWidth - self.blockLineWidth,
                                                     self.yPos - self.boardLineWidth - self.blockLineWidth,
                                                     (self.blockSize * self.colNum) + (2 * self.boardLineWidth) + (
                                                             2 * self.blockLineWidth), self.boardLineWidth], 0)
        pygame.draw.rect(gameDisplay, BORDER_COLOR, [self.xPos + (self.blockSize * self.colNum) + self.blockLineWidth,
                                                     self.yPos - self.boardLineWidth - self.blockLineWidth,
                                                     self.boardLineWidth,
                                                     (self.blockSize * self.rowNum) + (2 * self.boardLineWidth) + (
                                                             2 * self.blockLineWidth)], 0)
        pygame.draw.rect(gameDisplay, BORDER_COLOR, [self.xPos - self.boardLineWidth - self.blockLineWidth,
                                                     self.yPos - self.boardLineWidth - self.blockLineWidth,
                                                     self.boardLineWidth,
                                                     (self.blockSize * self.rowNum) + (2 * self.boardLineWidth) + (
                                                             2 * self.blockLineWidth)], 0)
        pygame.draw.rect(gameDisplay, BORDER_COLOR, [self.xPos - self.boardLineWidth - self.blockLineWidth,
                                                     self.yPos + (self.blockSize * self.rowNum) + self.blockLineWidth,
                                                     (self.blockSize * self.colNum) + (2 * self.boardLineWidth) + (
                                                             2 * self.blockLineWidth), self.boardLineWidth], 0)

    def draw_GAMEBOARD_CONTENT(self):

        if self.gameStatus == 'firstStart':

            titleText = fontTitle.render('TETRIS', False, WHITE)
            gameDisplay.blit(titleText, (self.xPos + +1.55 * self.blockSize, self.yPos + 8 * self.blockSize))

            versionText = fontVersion.render('v 1.0', False, WHITE)
            gameDisplay.blit(versionText, (self.xPos + +7.2 * self.blockSize, self.yPos + 11.5 * self.blockSize))

        else:

            for row in range(0, self.rowNum):
                for col in range(0, self.colNum):
                    if self.blockMat[row][col] == 'empty':
                        self.erase_BLOCK(self.xPos, self.yPos, row, col)
                    else:
                        self.draw_BLOCK(self.xPos, self.yPos, row, col, blockColors[self.blockMat[row][col]])

            if self.piece.status == 'moving':
                for i in range(0, 4):
                    self.draw_BLOCK(self.xPos, self.yPos, self.piece.blocks[i].currentPos.row,
                                    self.piece.blocks[i].currentPos.col, blockColors[self.piece.type])

            if self.gamePause == True:
                pygame.draw.rect(gameDisplay, DARK_GRAY,
                                 [self.xPos + 1 * self.blockSize, self.yPos + 8 * self.blockSize, 8 * self.blockSize,
                                  4 * self.blockSize], 0)
                pauseText = fontPAUSE.render('PAUSE', False, BLACK)
                gameDisplay.blit(pauseText, (self.xPos + +1.65 * self.blockSize, self.yPos + 8 * self.blockSize))

            if self.gameStatus == 'gameOver':
                pygame.draw.rect(gameDisplay, LIGHT_GRAY,
                                 [self.xPos + 1 * self.blockSize, self.yPos + 8 * self.blockSize, 8 * self.blockSize,
                                  8 * self.blockSize], 0)
                gameOverText0 = fontGAMEOVER.render('GAME', False, BLACK)
                gameDisplay.blit(gameOverText0, (self.xPos + +2.2 * self.blockSize, self.yPos + 8 * self.blockSize))
                gameOverText1 = fontGAMEOVER.render('OVER', False, BLACK)
                gameDisplay.blit(gameOverText1, (self.xPos + +2.35 * self.blockSize, self.yPos + 12 * self.blockSize))

    def draw_SCOREBOARD_BORDER(self):
        pygame.draw.rect(gameDisplay, BORDER_COLOR, [self.xPos + (self.blockSize * self.colNum) + self.blockLineWidth,
                                                     self.yPos - self.boardLineWidth - self.blockLineWidth,
                                                     self.scoreBoardWidth + self.boardLineWidth, self.boardLineWidth],
                         0)
        pygame.draw.rect(gameDisplay, BORDER_COLOR, [self.xPos + (
                self.blockSize * self.colNum) + self.boardLineWidth + self.blockLineWidth + self.scoreBoardWidth,
                                                     self.yPos - self.boardLineWidth - self.blockLineWidth,
                                                     self.boardLineWidth,
                                                     (self.blockSize * self.rowNum) + (2 * self.boardLineWidth) + (
                                                             2 * self.blockLineWidth)], 0)
        pygame.draw.rect(gameDisplay, BORDER_COLOR, [self.xPos + (self.blockSize * self.colNum) + self.blockLineWidth,
                                                     self.yPos + (self.blockSize * self.rowNum) + self.blockLineWidth,
                                                     self.scoreBoardWidth + self.boardLineWidth, self.boardLineWidth],
                         0)

    def draw_SCOREBOARD_CONTENT(self):

        xPosRef = self.xPos + (self.blockSize * self.colNum) + self.boardLineWidth + self.blockLineWidth
        yPosRef = self.yPos
        yLastBlock = self.yPos + (self.blockSize * self.rowNum)

        if self.gameStatus == 'running':
            nextPieceText = fontSB.render('next:', False, TEXT_COLOR)
            gameDisplay.blit(nextPieceText, (xPosRef + self.blockSize, self.yPos))

            blocks = [[0, 0], [0, 0], [0, 0], [0, 0]]
            origin = [0, 0]
            for i in range(0, 4):
                blocks[i][ROW] = origin[ROW] + pieceDefs[self.nextPieces[1]][i][ROW]
                blocks[i][COL] = origin[COL] + pieceDefs[self.nextPieces[1]][i][COL]

                if self.nextPieces[1] == 'O':
                    self.draw_BLOCK(xPosRef + 0.5 * self.blockSize, yPosRef + 2.25 * self.blockSize, blocks[i][ROW],
                                    blocks[i][COL], blockColors[self.nextPieces[1]])
                elif self.nextPieces[1] == 'I':
                    self.draw_BLOCK(xPosRef + 0.5 * self.blockSize, yPosRef + 1.65 * self.blockSize, blocks[i][ROW],
                                    blocks[i][COL], blockColors[self.nextPieces[1]])
                else:
                    self.draw_BLOCK(xPosRef + 1 * self.blockSize, yPosRef + 2.25 * self.blockSize, blocks[i][ROW],
                                    blocks[i][COL], blockColors[self.nextPieces[1]])

            if self.gamePause == False:
                pauseText = fontSmall.render('P -> pause', False, WHITE)
                gameDisplay.blit(pauseText, (xPosRef + 1 * self.blockSize, yLastBlock - 15 * self.blockSize))
            else:
                unpauseText = fontSmall.render('P -> unpause', False, self.whiteSineAnimation())
                gameDisplay.blit(unpauseText, (xPosRef + 1 * self.blockSize, yLastBlock - 15 * self.blockSize))

            restartText = fontSmall.render('R -> restart', False, WHITE)
            gameDisplay.blit(restartText, (xPosRef + 1 * self.blockSize, yLastBlock - 14 * self.blockSize))

        else:

            yBlockRef = 0.3
            text0 = fontSB.render('press', False, self.whiteSineAnimation())
            gameDisplay.blit(text0, (xPosRef + self.blockSize, self.yPos + yBlockRef * self.blockSize))
            text1 = fontSB.render('enter', False, self.whiteSineAnimation())
            gameDisplay.blit(text1, (xPosRef + self.blockSize, self.yPos + (yBlockRef + 1.5) * self.blockSize))
            text2 = fontSB.render('to', False, self.whiteSineAnimation())
            gameDisplay.blit(text2, (xPosRef + self.blockSize, self.yPos + (yBlockRef + 3) * self.blockSize))
            if self.gameStatus == 'firstStart':
                text3 = fontSB.render('start', False, self.whiteSineAnimation())
                gameDisplay.blit(text3, (xPosRef + self.blockSize, self.yPos + (yBlockRef + 4.5) * self.blockSize))
            else:
                text3 = fontSB.render('restart', False, self.whiteSineAnimation())
                gameDisplay.blit(text3, (xPosRef + self.blockSize, self.yPos + (yBlockRef + 4.5) * self.blockSize))

        pygame.draw.rect(gameDisplay, BORDER_COLOR,
                         [xPosRef, yLastBlock - 12.5 * self.blockSize, self.scoreBoardWidth, self.boardLineWidth], 0)

        scoreText = fontSB.render('score:', False, TEXT_COLOR)
        gameDisplay.blit(scoreText, (xPosRef + self.blockSize, yLastBlock - 12 * self.blockSize))
        scoreNumText = fontSB.render(str(self.score), False, NUM_COLOR)
        gameDisplay.blit(scoreNumText, (xPosRef + self.blockSize, yLastBlock - 10 * self.blockSize))

        levelText = fontSB.render('level:', False, TEXT_COLOR)
        gameDisplay.blit(levelText, (xPosRef + self.blockSize, yLastBlock - 8 * self.blockSize))
        levelNumText = fontSB.render(str(self.level), False, NUM_COLOR)
        gameDisplay.blit(levelNumText, (xPosRef + self.blockSize, yLastBlock - 6 * self.blockSize))

        linesText = fontSB.render('lines:', False, TEXT_COLOR)
        gameDisplay.blit(linesText, (xPosRef + self.blockSize, yLastBlock - 4 * self.blockSize))
        linesNumText = fontSB.render(str(self.lines), False, NUM_COLOR)
        gameDisplay.blit(linesNumText, (xPosRef + self.blockSize, yLastBlock - 2 * self.blockSize))

    # All the screen drawings occurs in this function, called at each game loop iteration
    def draw(self):

        self.draw_GAMEBOARD_BORDER()
        self.draw_SCOREBOARD_BORDER()

        self.draw_GAMEBOARD_CONTENT()
        self.draw_SCOREBOARD_CONTENT()

    def whiteSineAnimation(self):

        sine = math.floor(255 * math.fabs(math.sin(2 * math.pi * (gameClock.frameTick / (SINE_ANI_PERIOD * 2)))))
        # sine = 127 + math.floor(127 * math.sin(2*math.pi*(gameClock.frameTick/SINE_ANI_PERIOD)))
        sineEffect = [sine, sine, sine]
        return sineEffect

    def lineClearAnimation(self):

        clearAniStage = math.floor((gameClock.frameTick - gameClock.clearAniStart) / CLEAR_ANI_PERIOD)
        halfCol = math.floor(self.colNum / 2)
        if clearAniStage < halfCol:
            for i in range(0, 4):
                if self.clearedLines[i] >= 0:
                    self.blockMat[self.clearedLines[i]][(halfCol) + clearAniStage] = 'empty'
                    self.blockMat[self.clearedLines[i]][(halfCol - 1) - clearAniStage] = 'empty'
        else:
            self.lineClearStatus = 'cleared'

    def dropFreeBlocks(self):  # Drops down the floating blocks after line clears occur

        for cLIndex in range(0, 4):
            if self.clearedLines[cLIndex] >= 0:
                for rowIndex in range(self.clearedLines[cLIndex], 0, -1):
                    for colIndex in range(0, self.colNum):
                        self.blockMat[rowIndex + cLIndex][colIndex] = self.blockMat[rowIndex + cLIndex - 1][colIndex]

                for colIndex in range(0, self.colNum):
                    self.blockMat[0][colIndex] = 'empty'

    def getCompleteLines(self):  # Returns index list(length of 4) of cleared lines(-1 if not assigned as cleared line)

        clearedLines = [-1, -1, -1, -1]
        cLIndex = -1
        rowIndex = self.rowNum - 1

        while rowIndex >= 0:
            for colIndex in range(0, self.colNum):
                if self.blockMat[rowIndex][colIndex] == 'empty':
                    rowIndex = rowIndex - 1
                    break
                if colIndex == self.colNum - 1:
                    cLIndex = cLIndex + 1
                    clearedLines[cLIndex] = rowIndex
                    rowIndex = rowIndex - 1

        if cLIndex >= 0:
            gameClock.clearAniStart = gameClock.frameTick
            self.lineClearStatus = 'clearRunning'
        else:
            self.prepareNextSpawn()

        return clearedLines

    def prepareNextSpawn(self):
        self.generateNextPiece()
        self.lineClearStatus = 'idle'
        self.piece.status = 'uncreated'

    def generateNextTwoPieces(self):
        self.nextPieces[0] = pieceNames[random.randint(0, 6)]
        self.nextPieces[1] = pieceNames[random.randint(0, 6)]
        self.piece.type = self.nextPieces[0]

    def generateNextPiece(self):
        self.nextPieces[0] = self.nextPieces[1]
        self.nextPieces[1] = pieceNames[random.randint(0, 6)]
        self.piece.type = self.nextPieces[0]

    def checkAndApplyGameOver(self):
        if self.piece.gameOverCondition == True:
            self.gameStatus = 'gameOver'
            for i in range(0, 4):
                if self.piece.blocks[i].currentPos.row >= 0 and self.piece.blocks[i].currentPos.col >= 0:
                    self.blockMat[self.piece.blocks[i].currentPos.row][
                        self.piece.blocks[i].currentPos.col] = self.piece.type

    def updateScores(self):

        clearedLinesNum = 0
        for i in range(0, 4):
            if self.clearedLines[i] > -1:
                clearedLinesNum = clearedLinesNum + 1

        self.score = self.score + (self.level + 1) * baseLinePoints[clearedLinesNum] + self.piece.dropScore
        if self.score > 999999:
            self.score = 999999
        self.lines = self.lines + clearedLinesNum
        self.level = STARTING_LEVEL + math.floor(self.lines / 10)
        if self.level > 99:
            self.level = 99

    def updateSpeed(self):

        if self.level < 29:
            gameClock.fall.framePeriod = levelSpeeds[self.level]
        else:
            gameClock.fall.framePeriod = 1

        if gameClock.fall.framePeriod < 4:
            gameClock.fall.framePeriod = gameClock.move.framePeriod

    # All the game events and mechanics are placed in this function, called at each game loop iteration
    def gameAction(self):

        if self.gameStatus == 'firstStart':
            if key.enter.status == 'pressed':
                self.restart()

        elif self.gameStatus == 'running':

            if key.restart.trig == True:
                self.restart()
                key.restart.trig = False

            if self.gamePause == False:

                self.piece.move(self.blockMat)
                self.checkAndApplyGameOver()

                if key.pause.trig == True:
                    gameClock.pause()
                    self.gamePause = True
                    key.pause.trig = False

                if self.gameStatus != 'gameOver':
                    if self.piece.status == 'moving':
                        if key.rotate.trig == True:
                            self.piece.rotate('CW')
                            key.rotate.trig = False

                        # HardDrop is done over the block mat?

                        if key.cRotate.trig == True:
                            self.piece.rotate('cCW')
                            key.cRotate.trig = False

                    elif self.piece.status == 'collided':
                        if self.lineClearStatus == 'idle':
                            for i in range(0, 4):
                                self.blockMat[self.piece.blocks[i].currentPos.row][
                                    self.piece.blocks[i].currentPos.col] = self.piece.type
                            self.clearedLines = self.getCompleteLines()
                            self.updateScores()
                            self.updateSpeed()
                        elif self.lineClearStatus == 'clearRunning':
                            self.lineClearAnimation()
                        else:  # 'clearFin'
                            self.dropFreeBlocks()
                            self.prepareNextSpawn()

            else:  # self.gamePause = False
                if key.pause.trig == True:
                    # TODO Maybe permanently pause the game?
                    gameClock.unpause()
                    self.gamePause = False
                    key.pause.trig = False

        else:  # 'gameOver'
            if key.enter.status == 'pressed':
                self.restart()


Moving Piece Class: Describes the piece which is currently in play

* Added Functions here which allow us to move the piece automatically, whilst staying within the bounds of the game

In [ ]:
# Class for all the definitions of current moving piece
class MovingPiece:

    def __init__(self, colNum, rowNum, status):

        self.colNum = colNum
        self.rowNum = rowNum

        # Save the orientation of the piece (always a modulo of 4)
        self.rotation_orientation = 0

        self.blockMat = [['empty'] * colNum for i in range(rowNum)]

        self.blocks = []
        for i in range(0, 4):
            self.blocks.append(MovingBlock())

        self.currentDef = [[0] * 2 for i in range(4)]
        self.status = status  # 'uncreated' 'moving' 'collided'
        self.type = 'I'  # 'O', 'T', 'S', 'Z', 'J', 'L'

        self.gameOverCondition = False

        self.dropScore = 0
        self.lastMoveType = 'noMove'

    def get_blocks(self):
        return self.blocks

    def get_type(self):
        return self.type

    def get_status(self):
        return self.status

    def applyNextMove(self):
        for i in range(0, 4):
            self.blocks[i].currentPos.col = self.blocks[i].nextPos.col
            self.blocks[i].currentPos.row = self.blocks[i].nextPos.row

    # move the blocks a certain amount in a given direction (right = col+, left = col-, down = row+)
    def applyMove(self, row_move, col_move):
        for i in range(0, 4):
            self.blocks[i].currentPos.col += col_move
            self.blocks[i].currentPos.row += row_move

    def applyFastMove(self):

        if gameClock.move.check(gameClock.frameTick) == True:
            if self.lastMoveType == 'downRight' or self.lastMoveType == 'downLeft' or self.lastMoveType == 'down':
                self.dropScore = self.dropScore + 1
            self.applyNextMove()

    def slowMoveAction(self):

        if gameClock.fall.check(gameClock.frameTick) == True:
            if self.movCollisionCheck('down') == True:
                self.createNextMove('noMove')
                self.status = 'collided'
            else:
                self.createNextMove('down')
                self.applyNextMove()

    def set_collided(self):
        self.status = 'collided'

    def createNextMove(self, moveType):

        self.lastMoveType = moveType

        for i in range(0, 4):
            self.blocks[i].nextPos.row = self.blocks[i].currentPos.row + directions[moveType][ROW]
            self.blocks[i].nextPos.col = self.blocks[i].currentPos.col + directions[moveType][COL]

    def movCollisionCheck_BLOCK(self, dirType, blockIndex):
        if dirType == 'down':
            if (self.blocks[blockIndex].currentPos.row + 1 > self.rowNum - 1) or \
                    self.blockMat[self.blocks[blockIndex].currentPos.row + directions[dirType][ROW]][
                        self.blocks[blockIndex].currentPos.col + directions[dirType][COL]] != 'empty':
                return True
        else:
            if (((directions[dirType][COL]) * (self.blocks[blockIndex].currentPos.col + directions[dirType][COL])) > (
                    ((self.colNum - 1) + (directions[dirType][COL]) * (self.colNum - 1)) / 2) or
                    self.blockMat[self.blocks[blockIndex].currentPos.row + directions[dirType][ROW]][
                        self.blocks[blockIndex].currentPos.col + directions[dirType][COL]] != 'empty'):
                return True
        return False

    def movCollisionCheck(self, dirType):  # Collision check for next move
        for i in range(0, 4):
            if self.movCollisionCheck_BLOCK(dirType, i) == True:
                return True
        return False

    def rotCollisionCheck_BLOCK(self, blockCoor):
        if (blockCoor[ROW] > self.rowNum - 1 or blockCoor[ROW] < 0 or blockCoor[COL] > self.colNum - 1 or blockCoor[
            COL] < 0 or self.blockMat[blockCoor[ROW]][blockCoor[COL]] != 'empty'):
            return True
        return False

    def rotCollisionCheck(self, blockCoorList):  # Collision check for rotation
        for i in range(0, 4):
            if self.rotCollisionCheck_BLOCK(blockCoorList[i]) == True:
                return True
        return False

    def spawnCollisionCheck(self, origin):  # Collision check for spawn

        for i in range(0, 4):
            spawnRow = origin[ROW] + pieceDefs[self.type][i][ROW]
            spawnCol = origin[COL] + pieceDefs[self.type][i][COL]
            if spawnRow >= 0 and spawnCol >= 0:
                if self.blockMat[spawnRow][spawnCol] != 'empty':
                    return True
        return False

    def findOrigin(self):

        origin = [0, 0]
        origin[ROW] = self.blocks[0].currentPos.row - self.currentDef[0][ROW]
        origin[COL] = self.blocks[0].currentPos.col - self.currentDef[0][COL]
        return origin

    def rotate(self, rotationType):

        if self.type != 'O':
            tempBlocks = [[0] * 2 for i in range(4)]
            origin = self.findOrigin()

            if self.type == 'I':
                pieceMatSize = 4
            else:
                pieceMatSize = 3

            for i in range(0, 4):
                if rotationType == 'CW':
                    tempBlocks[i][ROW] = origin[ROW] + self.currentDef[i][COL]
                    tempBlocks[i][COL] = origin[COL] + (pieceMatSize - 1) - self.currentDef[i][ROW]
                else:
                    tempBlocks[i][COL] = origin[COL] + self.currentDef[i][ROW]
                    tempBlocks[i][ROW] = origin[ROW] + (pieceMatSize - 1) - self.currentDef[i][COL]

            if self.rotCollisionCheck(tempBlocks) == False:
                for i in range(0, 4):
                    self.blocks[i].currentPos.row = tempBlocks[i][ROW]
                    self.blocks[i].currentPos.col = tempBlocks[i][COL]
                    self.currentDef[i][ROW] = self.blocks[i].currentPos.row - origin[ROW]
                    self.currentDef[i][COL] = self.blocks[i].currentPos.col - origin[COL]

    def spawn(self):

        self.dropScore = 0

        origin = [0, 3]

        for i in range(0, 4):
            self.currentDef[i] = list(pieceDefs[self.type][i])

        spawnTry = 0
        while spawnTry < 2:
            if self.spawnCollisionCheck(origin) == False:
                break
            else:
                spawnTry = spawnTry + 1
                origin[ROW] = origin[ROW] - 1
                self.gameOverCondition = True
                self.status = 'collided'

        for i in range(0, 4):
            spawnRow = origin[ROW] + pieceDefs[self.type][i][ROW]
            spawnCol = origin[COL] + pieceDefs[self.type][i][COL]
            self.blocks[i].currentPos.row = spawnRow
            self.blocks[i].currentPos.col = spawnCol

    def move(self, lastBlockMat):

        if self.status == 'uncreated':
            self.status = 'moving'
            self.blockMat = lastBlockMat
            self.spawn()

        elif self.status == 'moving':

            if key.down.status == 'pressed':
                if key.xNav.status == 'right':
                    if self.movCollisionCheck('down') == True:
                        self.createNextMove('noMove')
                        self.status = 'collided'
                    elif self.movCollisionCheck('downRight') == True:
                        self.createNextMove('down')
                    else:
                        self.createNextMove('downRight')

                elif key.xNav.status == 'left':
                    if self.movCollisionCheck('down') == True:
                        self.createNextMove('noMove')
                        self.status = 'collided'
                    elif self.movCollisionCheck('downLeft') == True:
                        self.createNextMove('down')
                    else:
                        self.createNextMove('downLeft')

                else:  # 'idle'
                    if self.movCollisionCheck('down') == True:
                        self.createNextMove('noMove')
                        self.status = 'collided'
                    else:
                        self.createNextMove('down')

                self.applyFastMove()

            elif key.down.status == 'idle':
                if key.xNav.status == 'right':
                    if self.movCollisionCheck('right') == True:
                        self.createNextMove('noMove')
                    else:
                        self.createNextMove('right')
                elif key.xNav.status == 'left':
                    if self.movCollisionCheck('left') == True:
                        self.createNextMove('noMove')
                    else:
                        self.createNextMove('left')
                else:
                    self.createNextMove('noMove')

                self.applyFastMove()

                self.slowMoveAction()

            else:  # 'released'
                key.down.status = 'idle'
            # gameClock.fall.preFrame = gameClock.frameTick #Commented out because each seperate down key press and release creates a delay which makes the game easier

    # else: # 'collided'

    
    
    
    # AI Assistance functions

    # __________________________

    # attempt to move current piece to the left, if possible: move and return true, else: return false
    def move_piece_left(self):
        if self.movCollisionCheck('left') == True:
            self.createNextMove('noMove')
            self.applyNextMove()
            return False
        else:
            self.createNextMove('left')
            self.applyNextMove()
            return True

    # attempt to move current piece to the right, if possible: move and return true, else: return false
    def move_piece_right(self):
        if self.movCollisionCheck('right') == True:
            self.createNextMove('noMove')
            self.applyNextMove()
            return False
        else:
            self.createNextMove('right')
            self.applyNextMove()
            return True

    # attempt to rotate the piece, if possible: rotate and return true, else: return false
    def rotate_piece(self):

        tempBlocks = [[0] * 2 for i in range(4)]
        origin = self.findOrigin()

        if self.type == 'I':
            pieceMatSize = 4
        else:
            pieceMatSize = 3

        for i in range(0, 4):
            tempBlocks[i][ROW] = origin[ROW] + self.currentDef[i][COL]
            tempBlocks[i][COL] = origin[COL] + (pieceMatSize - 1) - self.currentDef[i][ROW]

        if self.rotCollisionCheck(tempBlocks) == False:
            self.rotate('CW')
            self.rotation_orientation += 1

            # by using modulo here, the same number will always refer to the same rotation orientation
            self.rotation_orientation = self.rotation_orientation % 4
            return True
        else:
            return False

    # attempt to move the piece down, if possible: move and return true, else: return false
    def move_piece_down(self):
        if self.movCollisionCheck('down') == True:
            self.createNextMove('noMove')
            self.status = 'collided'
            self.applyNextMove()
            return False
        else:
            self.createNextMove('down')
            self.applyNextMove()
            return True

    # returns the positions of the moving piece block as an 1D array of position [block1x, block1y, block2x, block2y, ....]
    def get_block_positions(self):

        block_positions = []

        for block in self.blocks:
            block_positions.append(block.currentPos.row)
            block_positions.append(block.currentPos.col)

        return np.array(block_positions)




In [ ]:
# Class for the blocks of the moving piece. Each piece is made of 4 blocks in Tetris game
class MovingBlock:

    def __init__(self):
        self.currentPos = self.CurrentPosClass(0, 0)
        self.nextPos = self.NextPosClass(0, 0)

    class CurrentPosClass:

        def __init__(self, row, col):
            self.row = row
            self.col = col

    class NextPosClass:

        def __init__(self, row, col):
            self.row = row
            self.col = col



The environment is now set up, and functions/methods were added which allow the agent to interact with the environment

We now need to create assistance functions to allow the agent to evaluate the environment


Aggregate Height
Holes
Line Clears
Bumpiness

Move score = - a * ah + -b * holes + c * LC + - d * b

In [ ]:
# AI Assistance Function
# _________________________________________________________________________


# returns a matrix representation of the playing board
def get_Blockmat(mainBoard):
    # Init 2 Dimensional array
    blockMat = [[0 for c in range(10)] for r in range(20)]

    # Get the current blockMat
    for rows in range(len(mainBoard.blockMat)):
        for columns in range(len(mainBoard.blockMat[rows])):

            # if position is empty, insert a 0, else insert dict. value
            if mainBoard.get_mainBoard_pos(rows, columns) == "empty":
                blockMat[rows][columns] = 0
            else:
                blockMat[rows][columns] = pieceValue[mainBoard.get_mainBoard_pos(rows, columns)]

    # insert the moving block into the blockMat
    for blocks in mainBoard.piece.blocks:
        blockMat[blocks.currentPos.row][blocks.currentPos.col] = pieceValue[mainBoard.piece.type]

    # return the blockmat
    return np.matrix(blockMat)

In [ ]:
# returns the blockMat, but with only the moving piece
def get_Blockmat_onlyPiece(mainBoard):
    # Init 2 Dimensional array
    blockMat = [[0 for c in range(10)] for r in range(20)]

    # insert the moving block into the blockMat
    for blocks in mainBoard.piece.blocks:
        blockMat[blocks.currentPos.row][blocks.currentPos.col] = pieceValue[mainBoard.piece.get_type()]

    # return the blockMat
    return np.matrix(blockMat)


In [ ]:
# returns the current Blockmat without the piece that is in play
def get_Blockmat_noPiece(mainBoard):
    # Init 2 Dimensional array
    blockMat = [[0 for c in range(10)] for r in range(20)]

    # Get the current blockMat
    for rows in range(len(mainBoard.blockMat)):
        for columns in range(len(mainBoard.blockMat[rows])):

            # if position is empty, insert a 0, else insert dict. value
            if mainBoard.get_mainBoard_pos(rows, columns) == "empty":
                blockMat[rows][columns] = 0
            else:
                blockMat[rows][columns] = pieceValue[mainBoard.get_mainBoard_pos(rows, columns)]

    # return the blockMat
    return np.matrix(blockMat)


Neben der Evaluation Funktion die wichtigste Funktion im Programm. Diese Funktion erlaubt dem Agent eine vorhersage über den nächsten zustand der Umgebung zu machen

In [ ]:
# Function which returns the blockmat if the tetromino is dropped in the current position with the current rotation
def get_next_blockmat(mainBoard):
    # array which saves the distance from that block to the bottom
    distances = []

    # get blockMat WITHOUT (important) the moving piece
    blockMat = get_Blockmat_noPiece(mainBoard)

    # go through all blocks and find distance to the next block vertically
    # use the blockMat WITHOUT the moving piece for this, otherwise it will collide with itself.
    for blocks in mainBoard.piece.get_blocks():

        distance = 0
        next_block = blockMat[blocks.currentPos.row, blocks.currentPos.col]
        stopMovement = False

        # find the next block vertically or the bottom of the grid
        while next_block == 0 and stopMovement == False:

            next_block = blockMat[blocks.currentPos.row + distance, blocks.currentPos.col]

            if blocks.currentPos.row + distance >= 19:
                stopMovement = True

            elif next_block != 0:
                distance -= 1

            else:
                distance += 1

        distances.append(distance)

    # move all blocks down by the minimum distance found and insert into the blockMat
    minDistance = min(distances)

    for blocks in mainBoard.piece.blocks:
        blockMat[blocks.currentPos.row + minDistance, blocks.currentPos.col] = pieceValue[mainBoard.piece.type]

    return blockMat



In [ ]:
# get the minimum vertical distance to the next block
def get_vertical_distance(mainBoard):
    # array which saves the distance from that block to the bottom
    distances = []

    # get blockMat WITHOUT (important) the moving piece
    blockMat = get_Blockmat_noPiece(mainBoard)

    # go through all blocks and find distance to the next block vertically
    # use the blockMat WITHOUT the moving piece for this, otherwise it will collide with itself.
    for blocks in mainBoard.piece.blocks:

        distance = 0
        next_block = blockMat[blocks.currentPos.row, blocks.currentPos.col]
        stopMovement = False

        # find the next block vertically or the bottom of the grid
        while next_block == 0 and stopMovement == False:

            next_block = blockMat[blocks.currentPos.row + distance, blocks.currentPos.col]

            if blocks.currentPos.row + distance >= 19:
                stopMovement = True

            elif next_block != 0:
                distance -= 1

            else:
                distance += 1

        distances.append(distance)

    return min(distances)


In [ ]:
def reverse_blockmat(blockmat):
    reversed_blockmat = [['empty'] * 10 for i in range(20)]

    for rows in range(20):
        for cols in range(10):

            if blockmat[rows, cols] != 0:
                reversed_blockmat[rows][cols] = inv_pieceValue[blockmat[rows, cols]]

    return reversed_blockmat


Heuristic Auswertung

why did we choose these, what does each one mean?

In [ ]:
# returns the aggregate height of a blockMat
def calc_aggregate_height(blockMat):
    heights = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

    # go through every cell in the blockMat. As soon as number is found within a call the first time:
    # subtract the row index from 20 to obtain the height
    for rows in range(20):
        for columns in range(10):

            if blockMat[rows, columns] != 0 and heights[columns] == 0:
                heights[columns] = 20 - rows

    # Aggregate heights
    agg = 0
    for x in range(10):
        agg += heights[x]

    return agg



In [ ]:
# returns the sum of holes within a block mat
def calc_holes(blockMat):
    # array which saves if a column is covered
    covered = np.array([False, False, False, False, False, False, False, False, False, False])

    # array which saves the amount of holes in a column
    holes = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

    # Go through all cells of the blockMat
    # If a column is covered has a block in it, it will count as covered,
    # count all subsequent empty spaces in that column
    for rows in range(20):
        for columns in range(10):

            if blockMat[rows, columns] != 0:
                covered[columns] = True

            if blockMat[rows, columns] == 0 and covered[columns] == True:
                holes[columns] += 1

    # Aggregate holes
    agg = 0
    for x in range(10):
        agg += holes[x]

    return agg


In [ ]:
# returns the number of lines cleared with a move
def calc_lines_cleared(blockMat):
    # init clear lines variable
    lines_cleared = 0

    # go through all cells in the blockMat
    # if a column is completely filled, increase the number of lines cleared
    for rows in range(20):

        full_line = True

        for columns in range(10):

            if blockMat[rows, columns] == 0:
                full_line = False

        if full_line:
            lines_cleared += 1

    return lines_cleared


In [ ]:
# returns the bumpiness values of the blockMat
def calc_bumpiness(blockMat):
    heights = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

    # go through every cell in the blockMat. As soon as number is found within a call the first time:
    # subtract the row index from 20 to obtain the height
    for rows in range(20):
        for columns in range(10):

            if blockMat[rows, columns] != 0 and heights[columns] == 0:
                heights[columns] = 20 - rows

    bumpiness = 0

    # go through each value, and calculate the absolute difference in height to the adjacent column
    # only go to 9 to avoid index out of bounds
    for x in range(9):
        bumpiness += abs(heights[x] - heights[x + 1])

    return bumpiness


Evaluation Funktion

In [ ]:
# returns the score of the blockMat, based on the score heuristic and the given parameters
def eval_blockMat(a, b, c, d, blockMat):
    score_function = (a * calc_aggregate_height(blockMat)) + (b * calc_holes(blockMat)) + (
                c * calc_lines_cleared(blockMat)) + (d * calc_bumpiness(blockMat))
    return score_function




Der Agent kann nun mit der jetzigen position des Tetris blocks eine vorhersage über das Spiel machen, und den Reward ausrechnen. 

Wir müssen nun einen algorithmus erstellen, die den Reward von jeder block position errechnet und dan den block in die beste position fallen lässt.

Der best mögliche Algorithmus sieht wie folgt aus:

Hier waren mehrere probleme

* Index out of bounds ---> In game movement benutzen (was wurde davor benutzt)
    * Ungeplanter vorteil: Auch wenn der block nicht ganz nach links oder rechts kann, wertet er alle möglichen positionen aus....
    
* Frame tick... nur eine aktion pro frame tick des spielers, ansonsten wird die BlockMat nicht richtig upgedtated...


Wie haben wir das gelöst?
....

In [ ]:
def eval_sequence(mainboard, step, a, b, c, d):
    # move the piece to the very right
    if step < 10:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # go through each x position in rot 0, get the pos_value and save it
    elif step < 20:
        position_value = eval_blockMat(a, b, c, d, get_next_blockmat(mainboard))
        rotation = mainboard.piece.rotation_orientation
        position = mainboard.piece.get_block_positions()

        mainboard.piece.move_piece_right()

        return rotation, position, position_value, True, False

    # go back two steps (to give the piece space to rotate)
    elif step == 20 or step == 21:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # rotate piece
    elif step == 22:
        mainboard.piece.rotate_piece()

        return 0, 0, 0, False, False

    # move the piece back to the left
    elif step < 32:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # go through each position x in rot 1, get the pos_value and save it
    elif step < 42:
        position_value = eval_blockMat(a, b, c, d, get_next_blockmat(mainboard))
        rotation = mainboard.piece.rotation_orientation
        position = mainboard.piece.get_block_positions()

        mainboard.piece.move_piece_right()

        return rotation, position, position_value, True, False

    # go back two steps to allow for rotation
    elif step == 42 or step == 43:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # rotate the piece
    elif step == 44:
        mainboard.piece.rotate_piece()

        return 0, 0, 0, False, False

    # move piece back to the left
    elif step < 54:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # go through each position x in rot 2, get the pos_value and save it
    elif step < 64:
        position_value = eval_blockMat(a, b, c, d, get_next_blockmat(mainboard))
        rotation = mainboard.piece.rotation_orientation
        position = mainboard.piece.get_block_positions()

        mainboard.piece.move_piece_right()

        return rotation, position, position_value, True, False

    # go back two steps to allow rotation
    elif step == 64 or step == 65:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # rotate piece
    elif step == 66:
        mainboard.piece.rotate_piece()

        return 0, 0, 0, False, False

    # move piece back to left
    elif step < 76:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # go through each position x in rot 3, get the pos_value and save it
    elif step < 86:
        position_value = eval_blockMat(a, b, c, d, get_next_blockmat(mainboard))
        rotation = mainboard.piece.rotation_orientation
        position = mainboard.piece.get_block_positions()

        mainboard.piece.move_piece_right()

        return rotation, position, position_value, True, False

    # move the piece two steps back
    elif step == 86 or step == 87:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # rotate the piece
    elif step == 88:
        mainboard.piece.rotate_piece()

        return 0, 0, 0, False, False

    # move piece back to the left
    elif step < 98:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # after having gone through all positional steps, return the last value as true to show position sequence is over
    elif step == 98:

        return 0, 0, 0, False, True

In [ ]:
# Eval all positions of the potential next blockmat with the look ahead piece in play
def eval_next_blockMat(next_blockMat, next_pieces, a, b, c, d):
    # this function is called for every tuple of rotation and position of the current piece in play

    # Create a new mainboard object
    blockSize = 20
    boardColNum = 10
    boardRowNum = 20
    boardLineWidth = 10
    blockLineWidth = 1
    scoreBoardWidth = blockSize * (boardColNum // 2)
    boardPosX = DISPLAY_WIDTH * 0.3
    boardPosY = DISPLAY_HEIGHT * 0.15

    next_mainBoard = MainBoard(blockSize, boardPosX, boardPosY, boardColNum, boardRowNum, boardLineWidth,
                               blockLineWidth,
                               scoreBoardWidth)

    # set the blockmat of the new mainboard equal to the next blockmat
    next_mainBoard.set_blockmat(reverse_blockmat(next_blockMat))

    # set the piece of the new blockmat equal to the look ahead piece
    next_mainBoard.nextPieces = next_pieces

    # Store the achieved positional values in a list
    position_values = []

    # Analyse all the possible positions of this blockMat
    done = False
    step = 0
    while done is False:

        # We do not need to save rotation and position for this, as the AI will by its nature make that next move
        _, _, position_score, calculated, done = eval_sequence(next_mainBoard, step, a, b, c, d)

        # add the position score to the position_values
        if calculated:
            position_values.append(position_score)

        # increase the step
        step += 1

    # return the max score that could be achieved next step
    return max(position_values)


In [ ]:
# Eval Sequence with look ahead piece consideration
def eval_sequence_LookAhead(mainboard, step, a, b, c, d):
    # move the piece to the very right
    if step < 10:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # go through each x position in rot 0, get the pos_value and save it
    elif step < 20:
        position_value = eval_blockMat(a, b, c, d, get_next_blockmat(mainboard)) + eval_next_blockMat(get_next_blockmat(mainboard), mainboard.nextPieces, a, b, c, d)
        rotation = mainboard.piece.rotation_orientation
        position = mainboard.piece.get_block_positions()

        mainboard.piece.move_piece_right()

        return rotation, position, position_value, True, False

    # go back two steps (to give the piece space to rotate)
    elif step == 20 or step == 21:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # rotate piece
    elif step == 22:
        mainboard.piece.rotate_piece()

        return 0, 0, 0, False, False

    # move the piece back to the left
    elif step < 32:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # go through each position x in rot 1, get the pos_value and save it
    elif step < 42:
        position_value = eval_blockMat(a, b, c, d, get_next_blockmat(mainboard)) + eval_next_blockMat(get_next_blockmat(mainboard), mainboard.nextPieces, a, b, c, d)
        rotation = mainboard.piece.rotation_orientation
        position = mainboard.piece.get_block_positions()

        mainboard.piece.move_piece_right()

        return rotation, position, position_value, True, False

    # go back two steps to allow for rotation
    elif step == 42 or step == 43:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # rotate the piece
    elif step == 44:
        mainboard.piece.rotate_piece()

        return 0, 0, 0, False, False

    # move piece back to the left
    elif step < 54:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # go through each position x in rot 2, get the pos_value and save it
    elif step < 64:
        position_value = eval_blockMat(a, b, c, d, get_next_blockmat(mainboard)) + eval_next_blockMat(get_next_blockmat(mainboard), mainboard.nextPieces, a, b, c, d)
        rotation = mainboard.piece.rotation_orientation
        position = mainboard.piece.get_block_positions()

        mainboard.piece.move_piece_right()

        return rotation, position, position_value, True, False

    # go back two steps to allow rotation
    elif step == 64 or step == 65:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # rotate piece
    elif step == 66:
        mainboard.piece.rotate_piece()

        return 0, 0, 0, False, False

    # move piece back to left
    elif step < 76:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # go through each position x in rot 3, get the pos_value and save it
    elif step < 86:
        position_value = eval_blockMat(a, b, c, d, get_next_blockmat(mainboard)) + eval_next_blockMat(get_next_blockmat(mainboard), mainboard.nextPieces, a, b, c, d)
        rotation = mainboard.piece.rotation_orientation
        position = mainboard.piece.get_block_positions()

        mainboard.piece.move_piece_right()

        return rotation, position, position_value, True, False

    # move the piece two steps back
    elif step == 86 or step == 87:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # rotate the piece
    elif step == 88:
        mainboard.piece.rotate_piece()

        return 0, 0, 0, False, False

    # move piece back to the left
    elif step < 98:
        mainboard.piece.move_piece_left()

        return 0, 0, 0, False, False

    # after having gone through all positional steps, return the last value as true to show position sequence is over
    elif step == 98:

        return 0, 0, 0, False, True

Nun brauchen wir eine Funktkion die den block in die richtige position bringt....

In [ ]:
# position piece at optimal position
def position_at_max(mainboard, step, rotation, block_position):

    # opt_coords = (rotation, position from left)

    # move piece two steps right to allow rotation
    if step == 0 or step == 1:
        mainboard.piece.move_piece_right()
        
        return False

    # rotate the piece until it is in the correct position
    elif step < 6:
        if mainboard.piece.rotation_orientation != rotation:
            mainboard.piece.rotate_piece()
            
            return False
        if mainboard.piece.rotation_orientation == rotation:

            return False

    # move the piece back to the left
    elif step == 6 or step == 7:
        mainboard.piece.move_piece_left()
        
        return False

    # TODO Maybe return right if correct position cannot be achieved
    # Probably dont need to do that TO Do, because even if a block is blocking the whole right side, it will go through
    # All possible positions available to the piece in that situation

    # move piece to correct x position
    elif step < 20:
        if not np.array_equal(mainboard.piece.get_block_positions(), block_position):
            mainboard.piece.move_piece_right()

            return False

    else:
        
        return True



Jetzt setzten wir eine Episode des spiels mit dem gewählten agenten in form eines pygame gameLoops um...

In [ ]:
# let AI play game based on best position score
def AI_Game_loop(a, b, c, d):
    blockSize = 20
    boardColNum = 10
    boardRowNum = 20
    boardLineWidth = 10
    blockLineWidth = 1
    scoreBoardWidth = blockSize * (boardColNum // 2)
    boardPosX = DISPLAY_WIDTH * 0.3
    boardPosY = DISPLAY_HEIGHT * 0.15

    mainBoard = MainBoard(blockSize, boardPosX, boardPosY, boardColNum, boardRowNum, boardLineWidth, blockLineWidth,
                          scoreBoardWidth)

    xChange = 0

    gameExit = False

    # for each position we need to save the x position, the rotation and the score
    # for this we will use three arrays to make reading the information easier

    position_values = []
    rotation_mat = []
    position_mat = []

    step = 0
    done = False
    positioned = False

    print_info = True

    # Statistic trackers _ End of Game
    score_achieved = 0
    number_of_lines_cleared = 0
    single_line_clears = 0
    double_line_clears = 0
    triple_line_clears = 0
    quadruple_line_clears = 0
    frames_in_game = 0
    blocks_placed = 0

    # timed statistics _ take every 200 frames
    score_over_time = []
    aggregate_height_over_time = []
    bumpiness_over_time = []
    lines_cleared_over_time = []

    while not gameExit:  # Stay in this loop unless the game is quit

        if mainBoard.gameStatus == 'running':
            
            # Keep track of frame count (divided by FPS will give Time)
            frames_in_game += 1

            # Record timed statistics every 200 frames
            if gameClock.frameTick%200 == 0:
                score_over_time.append(mainBoard.score)
                aggregate_height_over_time.append(calc_aggregate_height(get_Blockmat_noPiece(mainBoard)))
                bumpiness_over_time.append(calc_bumpiness(get_Blockmat_noPiece(mainBoard)))
                lines_cleared_over_time.append(mainBoard.lines)


            key.enter.status = 'idle'

            # go through all positionary checks
            # eval_sequence return values: rotation, position from left, position_score, calculated, done
            if not done:
                
                rotation, position, position_score, calculated, done = eval_sequence_LookAhead(mainBoard, step, a, b, c, d)

                # if a position score was calculated, save the blocks attributes in the arrays
                if calculated:

                    debug_blockmat = get_Blockmat(mainBoard)
                    position_values.append(position_score)
                    rotation_mat.append(rotation)
                    position_mat.append(position)

                step += 1

            # if done position in the correct place
            if done and step == 99 and positioned == False:

                # reset step
                step = 0

                # get the optimal position coordinated
                max_value = max(position_values)
                max_index = position_values.index(max_value)

                opt_rotation = rotation_mat[max_index]
                opt_position = position_mat[max_index]


            elif done and positioned == False:
                if not position_at_max(mainBoard, step, opt_rotation, opt_position):
                    step += 1

                if position_at_max(mainBoard, step, opt_rotation, opt_position):
                    positioned = True

            # drop the piece
            elif done and positioned:

                # Debug Info
                if print_info == True:
                    
                    line_clears_this_move = calc_lines_cleared(get_next_blockmat(mainBoard))

                    if line_clears_this_move == 1:
                        single_line_clears += 1
                    elif line_clears_this_move == 2:
                        double_line_clears += 1
                    elif line_clears_this_move == 3:
                        triple_line_clears += 1
                    elif line_clears_this_move == 4:
                        quadruple_line_clears += 1

                    print_info = False

                if mainBoard.piece.status == 'moving':
                    mainBoard.piece.move_piece_down()

                else:
                    # piece is dropped, reset to start over with next block

                    blocks_placed += 1
                    position_values = []
                    rotation_mat = []
                    position_mat = []

                    done = False
                    positioned = False
                    print_info = True
                    step = 0

        # press enter if the game is in the first start
        if mainBoard.gameStatus == 'firstStart':

            key.enter.status = 'pressed'

        # When game over, return the game statistics needed statistics...
        if mainBoard.gameStatus == 'gameOver' or blocks_placed == 500:
            score_achieved = mainBoard.score
            number_of_lines_cleared = mainBoard.lines

            print("\n")
            print(f"SCORE ACHIEVED: {score_achieved}")
            print(f"CLEARED LINES:  {number_of_lines_cleared}")

            return score_achieved, number_of_lines_cleared, single_line_clears, double_line_clears, triple_line_clears, quadruple_line_clears, frames_in_game, blocks_placed, score_over_time, lines_cleared_over_time, aggregate_height_over_time, bumpiness_over_time

        # Events need to be obtained for display to be shown
        for event in pygame.event.get():
            if event.type == pygame.QUIT:  # Looks for quitting event in every iteration (Meaning closing the game window)
                gameExit = True

        gameDisplay.fill(BLACK)  # Whole screen is painted black in every iteration before any other drawings occur

        mainBoard.gameAction()  # Apply all the game actions here
        mainBoard.draw()  # Draw the new board after game the new game actions
        gameClock.update()  # Increment the frame tick

        pygame.display.update()  # Pygame display update
        clock.tick(200)  # Pygame clock tick function(200 fps) --> Quicker execution of the algorithm...

In [ ]:
# Go through a certain number of episodes with one agent and save the data in a csv file
def train_for_episodes(generation, agent, episodes):

    # Sum up all the scores to be able to get mean score of the agent
    sum_score = 0
    
    for episode in range(episodes):
        
        # Init pygame...
        pygame.init()
        pygame.font.init()

        DISPLAY_WIDTH = 800
        DISPLAY_HEIGHT = 600

        gameDisplay = pygame.display.set_mode((DISPLAY_WIDTH, DISPLAY_HEIGHT))
        pygame.display.set_caption('Tetris')
        clock = pygame.time.Clock()
        
        # Execute the episode and get the Data (Set up pygame parameters as well)
        score, lines_cleared, s_line_clears, d_line_clears, t_line_clears, q_line_clears, time, blocks_placed, score_over_time, lines_cleared_over_time, aggregate_height_over_time, bumpiness_over_time = AI_Game_loop(agent[1], agent[2], agent[3], agent[4]) 
            
        sum_score += score
        
        print(f"--- Episode {episode} Complete ---")
            
        # After finishing the episode, open csv file and save the data
        with open(f'Training_Data/TetBot_LookAhead_Gen_{generation}.csv', 'a', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)
            
            # insert episode description
            writer.writerow([f"Generation:"])
            writer.writerow([generation])
            writer.writerow([f"Agent:"])
            writer.writerow([agent[0]])
            writer.writerow([f"Parameters Used: {agent[1]}, {agent[2]}, {agent[3]}, {agent[4]}"])
            writer.writerow([f"Epsisode:"])
            writer.writerow([episode])
            
            # blank line (for formatting)
            writer.writerow(" ")
            
            # insert episode data
            writer.writerow([f"Score Achieved:"])
            writer.writerow([score])
            writer.writerow([f"Lines Cleared:"])
            writer.writerow([lines_cleared])
            writer.writerow([f"Single Line Clears:"])
            writer.writerow([s_line_clears])
            writer.writerow([f"Double Line Clears:"])
            writer.writerow([d_line_clears])
            writer.writerow([f"Triple Line Clears:"])
            writer.writerow([t_line_clears])
            writer.writerow([f"Quadruple Line Clears:"])
            writer.writerow([q_line_clears])
            
            # since we get the time in frames, and the game runs in 200 fps, to get seconds we divide time by 200
            writer.writerow([f"Episode Length: {time/200}s"])
            writer.writerow([f"Blocks Placed (Max 500): {blocks_placed}"])
            writer.writerow(" ")
            
            # timed statistics
            writer.writerow(["Timed Statistics taken every 200 in game frames"])
            writer.writerow(["Score Over Time"])
            writer.writerow(score_over_time)
            writer.writerow(["Lines Cleared Over Time"])
            writer.writerow(lines_cleared_over_time)
            writer.writerow(["Aggregate Height Over Time"])
            writer.writerow(aggregate_height_over_time)
            writer.writerow(["Bumpiness Over Time"])
            writer.writerow(bumpiness_over_time)
            writer.writerow(" ")
            
    mean_score = sum_score/episodes
    with open(f'Training_Data/TetBot_LookAhead_Gen_{generation}.csv', 'a', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)    
        writer.writerow([f"Agent Mean Score:"])
        writer.writerow([mean_score])
        writer.writerow(["----------------------------"])
        writer.writerow(" ")
    
    return sum_score/episodes


In [ ]:
# Initialise the first agents (randomly generated)

agent1 = [1, -0.16832757769902507, -0.07969240348619844, 0.5877047189074952, -0.9972892156845388]
agent2 = [2, -0.14954607467217684, -0.9774968347336302, 0.13427028912517958, -0.2656020416873196]
agent3 = [3, -0.8258790328448274, -0.8404444078192465, 0.53307541035142, -0.8005901004984177]
agent4 = [4, -0.9334359573824076, -0.045904963199957294, 0.6503535156893778, -0.6061473427950209]
agent5 = [5, -0.829727580586907, -0.41975582988738913, 0.89918899074433, -0.07084232044026251]
agent6 = [6, -0.13600271814013787, -0.253519858157485, 0.1962860915390705, -0.7289177596543426]
agent7 = [7, -0.8825819986662627, -0.12109975544550722, 0.4750358319413158, -0.842048199192208]
agent8 = [8, -0.6148155894252068, -0.13509881681925928, 0.3842068562457456, -0.6921409976087427]
agent9 = [9, -0.23616689030780524, -0.561639570397952, 0.9631285951239456, -0.10291717117694621]
agent10 = [10, -0.02833023645190702, -0.6992758306876569, 0.40806656324928814, -0.15988201219420206]

# Create agent list (generation)
agents = [agent1, agent2, agent3, agent4, agent5, agent6, agent7, agent8, agent9, agent10]

In [ ]:
# Generation Training

def train_generation(generation, agents, episodes):
    
    generation_results = []
    
    for agent in agents:
         
        # Let the agent run the episodes
        mean_score = train_for_episodes(generation, agent, episodes)
        generation_results.append(mean_score)
        
        print(f"---- Agent {agent[0]} Completed all Episodes ----")
        
    # After finishing the generation, add the generation results to the last row
    with open(f'Training_Data/TetBot_LookAhead_Gen_{generation}.csv', 'a', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(" ")
        writer.writerow(["--- AGENT MEAN SCORES ---"])
        writer.writerow(generation_results)
        writer.writerow(" ")
        writer.writerow(["Agents:"])
        writer.writerow(agents)
        writer.writerow(" ")
        writer.writerow(["-------------------------------------------"])
        writer.writerow(" ")
        
    return generation_results
 
# This works!

In [ ]:
# Tournament Selection and Mutation (10%)

In [ ]:
# Multiple Generation Training ---> delta < theta

Generation 1 ---> Tournament Selection ---> Breeding with Weighted Average ---> Mutation Factor ---> Generation 2 ---> Toleranz Wert Theta Theta

Nachdem alle Agenten 10 episoden gespielt haben nehmen wir von jedem die Average score um sie zu bewerten.

Nun muss sich die Population weiter entwickeln. Dazu benutzen wir tournament selection. Wir wählen zufällig 2 Parent indivuals aus der Population aus und erstellen aus diesen, basierend auf einem weighted average (für den stärkeren basierend auf prozent unterschied) einen neuen Agenten. Hier wird ein mutations factor von 10% mit einberechnet. Wenn dieser zutrifft wird ein zufälliges attribut des agenten bis zu +- 0.2 verändert. 

Das wiederholen wir ein zweites mal. 
Nun Löschen wir aus unserer Population die schwächsten 2 individuen, und setzten die zwei neuen ein. Dann Testen wir die Generation erneut. 

Dies wiederholen wir bis die differenz der änderungen zwischen den Generationen unter einen schellenwert theta fällt

In [ ]:
def create_new_Population(agents, gen_results):
    
    # Generate two random integers between 0 and 9 (Parent Individuals)
    random1 = random.randint(0, 9)
    random2 = random.randint(0, 9)
        
    # Make sure theser are not the same
    while random1 == random2:
        random2 = random.randint(0, 9)
        
    # Generate another two randome integers
    random3 = random.randint(0, 9)
    random4 = random.randint(0, 9)
    
    # Make sure this are uniqe as well
    while random3 == random1 or random3 == random2:
        random3 = random.randint(0, 9)
        
    while random4 == random1 or random4 == random2 or random4 == random3:
        random4 = random.randint(0, 9)
        
        
    print(random1, random2, random3, random4)
    # Take the two Agents and create the new offspring
    new_agent1 = create_offspring(agents[random1], gen_results[random1], agents[random2], gen_results[random2])
    new_agent2 = create_offspring(agents[random3], gen_results[random3], agents[random4], gen_results[random4])
    
    # Delete the two lowest scoring agents
    for x in range(2):
        
        del_index = gen_results.index(min(gen_results))
        
        temp_score = gen_results[del_index]
        temp_agent_tag = agents[del_index][0]
        
        del agents[del_index]
        del gen_results[del_index]
        
        print(f"Delete Agent {temp_agent_tag}, Agent scored: {temp_score}")
        
    # Append the new agents
    agents.append(new_agent1)
    agents.append(new_agent2)
    
    # Fix the agents tags
    tag = 1
    for agent in agents:
        agent[0] = tag
        tag += 1
        
    # return the agent list
    return agents
   
   
    
    

In [ ]:
# Create offpsring from two agents based on Weighted Average and mutation factor
def create_offspring(agent1, mean_agent1, agent2, mean_agent2 ,mutation_factor = 0.1):
    
    mutate = False
    
    print(agent1, mean_agent1)
    print(agent2, mean_agent2)
    
    # Does the offspring mutate?
    if np.random.random() < mutation_factor:
        mutate = True
    
    # init variables
    fittest = 0
    weighting = 0
    
    new_agent = []
    
    # Identify the stronger individual
    if mean_agent1 > mean_agent2:
        fittest = 1
    else:
        fittest = 2
        
    weight1 = 0
    weight2 = 0
        
    # Base weighting on the percentage difference between the mean values
    if fittest == 1:
        weighting = (mean_agent2/mean_agent1)
        weight1 = 1-weighting
        weight2 = weighting
        print(fittest, weight1, weight2)
        
    else:
        weighting = (mean_agent1/mean_agent2)
        weight1 = weighting
        weight2 = 1-weighting
        print(fittest, weight1, weight2)
        
    
    # Go through the parameters and adjust based on weighting
    for x in range(4):
            new_Parameter = (weight1 * agent1[x+1] + weight2 * agent2[x+1])
            new_agent.append(new_Parameter)
            
    # If mutated, add the mutation to one of the parameters and then normalize the vector
    if mutate:
        
        print("Mutation")
        
        # Select a random parameter
        mutation_index = random.randint(0,3)
        
        # Get the amount to be changed
        mutation = random.uniform(0., 0.2)
        
        # Apply the Mutation
        new_agent[mutation_index] += mutation
        
         # Normalize the Vector
        npAgent = np.array(new_agent)
        norm = np.linalg.norm(npAgent)
        norm_npAgent = npAgent/norm
        
        new_agent = norm_npAgent.tolist()
        
    # Add the Agent Tag to the beginning of the array and then return
    new_agent.insert(0, 11)
    
    return new_agent
    
        
        

### This is the Actual Optimization
The AI will continue to learn until a threshhold in the difference of results is not being passed anymore

In [ ]:
# Run Generations until Theta Value is Passed

def Optimize(agents, train_for_generations):
    
    generation = 0
    
    # Train one generation to get the initial results
    gen_results = train_generation(generation, agents, 10)
    
    # Adjust the population
    agents = create_new_Population(agents, gen_results)
    
    generation += 1
    
    for generations in range(train_for_generations):
        
        # Get results of the new generation
        new_results = train_generation(generation, agents, 10)
        
        #Adjust the population with the new scores
        agents = create_new_Population(agents, new_results)
            
        # Calculate delta
        sum_gen = 0
        sum_new = 0
        
        for x in range(len(gen_results)):
            sum_gen += gen_results[x]
            sum_new += new_results[x]
            
        delta = (sum_new - sum_gen)/sum_gen
        
        with open(f'Training_Data/TetBot_LookAhead_Gen_{generation}.csv', 'a', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(" ")
            writer.writerow("-------------")
            writer.writerow([f"Delta: {delta}"])
            
        gen_results = new_results
        generation += 1
    
    
    print(f"Trained for {train_for_generations} Generation....")
        
    return agents, gen_results
        
        
        

    

In [ ]:
# Run the Optimization ---> Train the AI

# Pygame Setup
clock = pygame.time.Clock()
key = GameKeyInput()
gameClock = GameClock()

# Optimize
new_agents, gen_results = Optimize(agents, 4)

In [ ]:
#-------------------------------------

# AGENT SAMPLE RUN

clock = pygame.time.Clock()
key = GameKeyInput()
gameClock = GameClock()
AI_Game_loop(-np.random.random(), -np.random.random(), np.random.random(), -np.random.random())


#-------------------------------------

In [ ]:
# -------------------------------

# GENERATION SAMPLE RUN
clock = pygame.time.Clock()
key = GameKeyInput()
gameClock = GameClock()

gen_results = train_generation(1, agents, 10)
print(gen_results)

# -------------------------------

In [ ]:
# SAMPLE RUN RESULT
gen_results = [332.0,2088.0,3960.0,1640.0,2560.0,444.0,2242.0,854.0,6268.0,10766.0]

In [ ]:
# SAMPLE NEW GENERATION
print(agents)
print(" ")
print(create_new_Population(agents, gen_results))

Daten Analyse Sehr wichtig:

Wie beeinfussen die Parameter die Spiel weise:

Beispiele aus den Daten nehmen!
e.g Gen 1 Agent 2 --> Vermeidet löcher sehr. ist Bumpiness eher egal. Baut daher sehr unebene felder die aber schnell mit einem einzelenen block weg gemacht werden können. Problem. Da auch höhe nicht stark gewichetet ist und line clears auch nicht, baut er sich sehr schnell ein. Kriegt jedoch hohe punktzahl weil er multi line clears bekommt!

Or the fitst agent for example which has a really high bumpiness weighting and everything else is low, so it simply try to create a smooth playing field....

These need to work in tandem to make the best agent

a too high reward on line clears will make you take the lines without wanting to go for a combo.... (immediate reward)

Exploration vs. Exploitation

It seems like a lower one here is actually better sometimes

Über dieses verhalten reden wenn die Heuristic erklärt wird. Sehr wichtig...

Durch alle Daten Gehen und diese erklären. 
Welche sache bevorzugen Multi_line clears etc....

Overtime Statistics to show that woukd be good:
Delta, Score, Lines on average per generation....

What Graphs to make? Update the generation overview of the agents with score and parameters etc...

In graphs show: Score over Generations, average time blocks placed per generation, theta (delta) change over generations...

Also look at how certain parameters prioritize multiline clears....


We should also track delta over time... see how the program learns. 

Also: We notice delta falling of substantially after a few generation. 
We come to the conclusion. Variance within this system is very high. Thusly it would make more sense to make fewer generations, but increase the episodes each agent gets to play, to compensate for this variance

Variance is high because the AI is learning to play with only on piece, no look ahead piece. Therefore the randomness of the game being played in this way has to be compensated for...

----

We ran the Agents for 15 Generations. Not really as successful as it should be. 
There was a rise in Delta increase for a few episodes, but this evened out relativly fast.
After episode 6, it seemed like a maximum was achieved. 

A few adjustments will be made:

Went from 10 episodes to 50 episodes per agent to account for the high variance within the environment
Increased the Frame rate to 1000

Made a change to the Fittness Calculation
Made line clears more important as aggregate height increases
Put it into google colab....

Google Collab - no video, remove display an draw functions...


---

Possibility to make the AI also take into account the lookahead piece. (to maybe negate the large variance

How to do it?

1. for each position and rotation of the current game. The AI checks how good the piece placement will be (immediate)

2. To make the game take into account the look ahead piece, we can take the possible blockmat that that rotation and position of the current piece would create, create a new mainboard with that blockmat (with the reverse blockmat function) and have the look ahead piece go through all possible positions. Get the highest score the lookahead piece could achieve with this blocmat, and add that to the score of the current piece

3. change the csv files to be easier readible in excel...




Interesting things: High priority for lowering height, and holes, but low prio for lines and bumpiness will cause it to make toweres. (But the beginning is always amazing, leaves very very little holes, could easily make quadra clears...

The cool thing about this AI is also, depending on what you choose to select as your fitness (criteria for elimination), the AI will begin to evolve to maximize that output...